In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import urllib
import logging
from dataclasses import dataclass
from collections import Counter

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
@dataclass
class CODE:
    SUCCESS:int=200

In [20]:
class Novel:
    def __init__(self,name,url) -> None:
        self.name=name
        self.url=url
        parse_res=urllib.parse.urlparse(self.url)
        self.domain=parse_res.scheme+"://"+parse_res.netloc
        self.Update()
        #TODO: async update

    def Update(self):
        raw_result=requests.get(self.url)
        if raw_result.status_code==CODE.SUCCESS:
            self.content=BeautifulSoup(raw_result.content)
        self.contentTable=self.GetContentTable()

    def GetContentTable(self):
        res=self.content.find_all("dd")
        counter=Counter(res)
        while(counter[res[0]]>1): # Remove duplicate newest chapters
            res.pop(0)
        res=list(map(lambda x:{"name":x.a.contents[0].strip(),"url": urllib.parse.urljoin(self.domain,x.a.attrs.get("href"))},res))
        return res
    def GetChapterRecord(self,idx):
        return self.contentTable[idx]
    def GetChapterPlain(self,idx):
        url=self.GetChapterRecord(idx)["url"]
        raw_res=requests.get(url)
        if raw_res.status_code==CODE.SUCCESS:
            raw_content=BeautifulSoup(raw_res.content)
        raw_page_content=raw_content.find_all("div",{"id":"content"})[0].find_all("p")
        raw_page_content_plain=map(lambda x:x.contents[0].strip(),raw_page_content)
        raw_page_content_plain=map(lambda x:x.replace("一秒记住www.9biqu.com","").strip(),raw_page_content_plain)
        raw_page_content_plain=map(lambda x:x.replace("首发尽在www.9biqu。com","").strip(),raw_page_content_plain)
        raw_page_content_plain=map(lambda x:x.replace("看小说上www,9biqu.com","").strip(),raw_page_content_plain)
        raw_page_content_filtered=list(filter(lambda x: not "天才一秒记住本站地址" in x.strip(),raw_page_content_plain))
        return raw_page_content_filtered

In [21]:
class NovelWebsite:
    def __init__(self,domain) -> None:
        self.domain=domain
    
    def search_url(self,keyword):
        return urllib.parse.urljoin(self.domain,f"search?keyword={urllib.parse.quote(keyword)}")
    
    def search_raw(self,keyword):
        data=requests.get(self.search_url(keyword))
        return data
    def join(self,url):
        return urllib.parse.urljoin(self.domain,url)

    def search_result(self,keyword):
        result=self.search_raw(keyword)
        logging.debug("STATUS_CODE: "+str(result.status_code))
        if result.status_code==CODE.SUCCESS:
            raw_page=BeautifulSoup(result.content)
            data=raw_page.find("div",{"id":"main"})
            return [{"name":data.a.contents[0].strip(),"url":self.join(data.a.attrs.get("href"))}]
    def GetNovel(self,keyword):
        books=self.search_result("灵境行者")
        novel=Novel(**books[0])
        return novel


In [22]:
# urllib.parse.urlparse(novel.url).netloc

In [23]:
biquge=NovelWebsite("https://www.9biqu.com/")
novel=biquge.GetNovel("灵境行者")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /search?keyword=%E7%81%B5%E5%A2%83%E8%A1%8C%E8%80%85 HTTP/1.1" 200 None
DEBUG:root:STATUS_CODE: 200
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /biquge/6027/ HTTP/1.1" 200 None


In [27]:
novel.GetChapterPlain(5)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /biquge/6027/15056957.html HTTP/1.1" 200 None


['艹.....张元清突然不想在主殿待下去了。',
 '他有种身处绝境，叫天天不应，叫地地不灵的惶恐。',
 '此处极端危险，可他却无法向外界求助，能依靠的只有自己。',
 '犹豫再三，他牙一咬心一横，俯身把裹着工服的骷髅拉了出来。',
 '哧拉～',
 '拉扯过程中，劳保服被轻易撕裂，过去那么多年，衣服早就朽烂。',
 '把尸体拖到烛光下，他忍着不适，开始检查起来。',
 '人虽然死了，但身体会说话，弄清楚对方的死因，能帮自己规避很多危险。',
 '“胸骨和肋骨断了几根，右肩有细密的裂纹，但不算严重.......”',
 '死者生前遭受过重创，但具体的死因，由于年代过于久远，看不出来。',
 '接着，张元清又从工人前辈的口袋里，找到了几张发黄发脆的纸，很有些年头了。',
 '纸上用楷体写着蝇头小字。',
 '张元清心里一喜，纸张明显是工人前辈在庙里找到的，这有助于他了解这座古庙的情况。',
 '借助烛光，他凝神阅读起纸张上的内容。',
 '“昨晚又有一位师弟失踪，这已经是庙里第三个离奇失踪的同门。。师兄弟们说，三道山闹了鬼，亦或者来了道行高深的妖精，每夜都要来庙里抓人吃，但庙里的诸多弟子修为伴身，师父更是方圆百里之内名声赫赫的真人，哪路妖精不开眼，敢来此处觅食。',
 '“至于鬼怪，我的镇尸符和唤灵符足以解决，都不用劳烦师父。我心里有不好的预感，得去寻大师兄合计合计..........”',
 '“今天又有一人失踪，这是第五人，师父让我们瞒着香客，不然会影响庙里的香火，他一定知道些什么。我和大师兄打算夜里巡逻.........”',
 '“三天过去了，期间又有三人失踪，可我和大师兄没有发现端倪，夜晚风平浪静。我心里不详的预感更深了........”',
 '“大师兄今天很奇怪，他似乎发现了什么，显得非常愤怒，我询问他，可他没有告诉我。他情绪不太好，我还是明天再询问吧。”',
 '“今天失踪的人是........大师兄，我找遍了整个三道山也没发现他，我，我受不了了，我要去找师父问个清楚，庙里的师兄弟们也支持我，因为他们也很害怕......”',
 '这段话的字迹有些潦草，可见笔记的主人心态有些崩溃了。',
 '张元清继续往下看：',
 '“经过我的一番追问，师父终于答应告诉我真相，我的预感没错，他果然知道师弟们离奇失踪的原因。但师父说，白日里人多嘴杂，太阳落山后他会